### pip install Wikipedia package to web scrape postal codes list

In [1]:
import pandas as pd

!pip install wikipedia
import wikipedia as wp

### Use pandas 'read_html' function to convert list to dataframe

In [2]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html, header = 0)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Check dataframe dimensions

In [3]:
df.shape

(180, 3)

### Excluded all instances of 'Not assigned' boroughs with .loc

In [4]:
df = df.loc[df["Borough"]!='Not assigned']
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### use the .shape method to print the number of rows of your dataframe

In [5]:
df.shape

(103, 3)